In [3]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);

In [4]:
pip install git+https://github.com/mimoralea/gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-req-build-1hc98_et
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-req-build-1hc98_et
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym_walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=34a5229834352b84d2cf43fa8203fd95f781383ddc6b1d8390244311e6e2c689
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ez4wptb/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym_walk


In [5]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [6]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [7]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [8]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [9]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [10]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [11]:
init_state

0

In [13]:
# obs, reward, terminated, truncated, info = env.step(RIGHT)
# done = terminated or truncated
# print("state:{0} - reward:{1} - done:{2} - info:{3}".format(obs, reward, done, info))

In [14]:
pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [15]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [16]:
# Create your own policy

pi_2 = lambda s: {
    0: RIGHT,
    1: RIGHT,
    2: RIGHT,
    3: DOWN,
    4: RIGHT,
    5: RIGHT,
    6: RIGHT,
    7: DOWN,
    8: RIGHT,
    9: RIGHT,
    10: RIGHT,
    11: DOWN,
    12: RIGHT,
    13: RIGHT,
    14: RIGHT,
    15: LEFT
}[s]


print("Name: Bharath Raj P")
print("Register Number: 212223230031")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)


Name: Bharath Raj P
Register Number: 212223230031
Policy:
| 00      > | 01      > | 02      > | 03      v |
| 04      > |           | 06      > |           |
| 08      > | 09      > | 10      > |           |
|           | 13      > | 14      > |           |


In [17]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))

Reaches goal 2.00%. Obtains an average undiscounted return of 0.0200.


In [19]:
print("Policy 1 (FrozenLake mapping):")
print("  Success Rate: {:.2f}%".format(probability_success(env, pi_frozenlake, goal_state=goal_state) * 100))
print("  Avg Return: {:.4f}".format(mean_return(env, pi_frozenlake)))

print("\nPolicy 2 :")
print("  Success Rate: {:.2f}%".format(probability_success(env, pi_2, goal_state=goal_state) * 100))
print("  Avg Return: {:.4f}".format(mean_return(env, pi_2)))

Policy 1 (FrozenLake mapping):
  Success Rate: 10.00%
  Avg Return: 0.1000

Policy 2 :
  Success Rate: 2.00%
  Avg Return: 0.0200


In [20]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)

    while True:
        delta = 0
        for s in range(len(P)):
            v = 0
            a = pi(s)
            for prob, next_state, reward, done in P[s][a]:
                v += prob * (reward + gamma * V[next_state] * (not done))
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return V

In [21]:
V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print_state_value_function(V1, P, n_cols=4, prec=5)

State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |


In [22]:
V2 = policy_evaluation(pi_2, P,gamma=0.99)
print_state_value_function(V2, P, n_cols=4, prec=5)

State-value function:
| 00 0.03462 | 01 0.03162 | 02 0.06419 | 03 0.03162 |
| 04 0.03868 |           | 06 0.09871 |           |
| 08 0.08259 | 09 0.2116 | 10 0.23494 |           |
|           | 13 0.40626 | 14 0.61323 |           |


In [23]:
V1>=V2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [30]:
if(np.sum(V1>=V2)>=11):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)>=11):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The first policy is the better policy
